In [8]:
!pip install tensorflow-datasets

In [9]:
# 개와 고양이 이미지 데이터 불러오기
import tensorflow_datasets as tfds

In [10]:
tfds.disable_progress_bar()

In [30]:
raw_train, raw_test = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[:20%]'],
    as_supervised=True
)

In [31]:
# FeaturesDict({
#     'image':Image(shape(None, None, 3), dtype=tf.unit8),
#     'image/filename':Text(shape=(), dtype=tf.string),
#     'label':ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
# })

In [32]:
# 개와 고양이 이미지 데이터 전처리
import numpy as np
import tensorflow as tf
from tensorflow.image import ResizeMethod

In [33]:
def preprocess(image, label):
    out_image = tf.image.resize(image,[96, 96], method=ResizeMethod.BICUBIC)
    out_image = tf.keras.applications.mobilenet_v2.preprocess_input(out_image)
    return out_image, label

In [34]:
batch_size = 32
train_batch = raw_train.map(preprocess).batch(batch_size)
test_batch = raw_test.map(preprocess).batch(batch_size)

In [37]:
# 사전 학습된 MobileNet 모델 로드
mobilenet_base = tf.keras.applications.MobileNetV2(
    input_shape=(96, 96, 3),
    weights="imagenet",
    include_top=False)

9406464/9406464 [==============================] - 4s 0us/step


In [38]:
# 사전 학습된 MoblieNet 모델 훈련 및 평가
mobilenet_base.trainable=False

mobilenet_model = tf.keras.Sequential([
    mobilenet_base,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1)
])

In [39]:
mobilenet_model.compile(optimizer='adam',
                        loss='binary_crossentropy',
                        metrics=['accuracy'])

mobilenet_model.fit(train_batch, epochs=5)
mobilenet_model.evaluate(test_batch, verbose=2)

Epoch 1/5
582/582 [==============================] - 81s 125ms/step - loss: 1.2045 - accuracy: 0.8908
Epoch 2/5
582/582 [==============================] - 71s 122ms/step - loss: 0.8384 - accuracy: 0.9306
Epoch 3/5
582/582 [==============================] - 72s 124ms/step - loss: 0.7233 - accuracy: 0.9409
Epoch 4/5
582/582 [==============================] - 76s 130ms/step - loss: 0.7641 - accuracy: 0.9389
Epoch 5/5
582/582 [==============================] - 72s 123ms/step - loss: 0.8611 - accuracy: 0.9336
146/146 - 21s - loss: 0.6650 - accuracy: 0.9469 - 21s/epoch - 141ms/step


[0.6650084257125854, 0.9469045400619507]